In [33]:
from torchaudio.models.decoder import ctc_decoder, cuda_ctc_decoder
import numpy as np 
import torch
from torch.nn.functional import softmax, log_softmax
from brainaudio.inference.inference_utils import _cer_and_wer
import torch.distributions as dist

In [248]:
language_model_path = "/data2/brain2text/lm/languageModel/"
lexicon_phonemes_file = f"{language_model_path}lexicon_phonemes.txt"
units_txt_file_pytorch = f"{language_model_path}units_pytorch.txt"

units_txt_file_pytorch_char = f"{language_model_path}units_pytorch_character.txt"
lexicon_char_file= f"{language_model_path}lexicon_char.txt"

imagineville_vocab = "/data2/brain2text/lm/vocab_lower_100k_pytorch.txt"

In [68]:
with open(units_txt_file_pytorch_char, 'r') as file:
    character_units = [line.strip() for line in file]

In [7]:
import pandas as pd
val_transcripts = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val.pkl")

In [8]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files("librispeech-4-gram")

/home3/ebrahim2/brainaudio/.venv/lib/python3.12/site-packages/torchaudio/models/decoder/_ctc_decoder.py:557: UserWarning: torchaudio.utils.download.download_asset has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  lexicon_file = download_asset(files.lexicon)
/home3/ebrahim2/brainaudio/.venv/lib/python3.12/site-packages/torchaudio/models/decoder/_ctc_decoder.py:558: UserWarning: torchaudio.utils.download.download_asset has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  tokens_file = download_asset(files.tokens)
/home3/ebrahim2/brainaudio/.venv/lib/python3.12/site-packages/torchaudio/models/decoder/

In [239]:
decoder_char = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=imagineville_vocab, 
                       beam_size=100, nbest=1, lm="/data2/brain2text/lm/lm_dec19_huge_4gram.arpa")

In [240]:
model_logits_char = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_combined_lw_char/logits_val.npz")

In [246]:

idx = 400
T = 5
print("Transcript: ", val_transcripts[idx])
print("\n")

# 1. Get the original logits
single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)

# 4. Perform beam search on the NOISY logits (and apply temperature)
beam_search_char = decoder_char(single_trial_logits_char[0:1, :50] / T)

for i in range(len(beam_search_char[0])):
    beam_search_transcript_char = " ".join(beam_search_char[0][i].words).strip()
    print(beam_search_transcript_char)

Transcript:  friday afternoon at five thirty


vitt atria fave
vitt eternit fave
vitt afferent fave
vitt ator fave
vitt aren fave
vitt datron fave
vidor ator fave
vidor afferent fave
ahranat fave
vitt ator a fave
vitt afferent a fave
vitt aren a fave
vitt datron a fave
vitt atria five
vitt eternit five
videotron at five
ahranat five
vitt afferent five
vitt ator five
vitt aren five
vitt datron five
vitt ator athie
ator at five
vidor ator five
vitt afferent athie
vitt ator attie
afferent at five
vidor afferent five
vitt atria frie
vitt aren athie
vitt datron athie
aren at five
vitt afferent attie
datron at five
vitt eternit frie
vitt aren attie
vitt datron attie
vidor ator athie
videos ator five
videotron five
vidor afferent athie
vidor ator attie
vitt ator at five
videos afferent five
atria five
vidor afferent attie
vitt afferent at five
eternit five
vitt afferent frie
vitt aren at five
videos ator athie
vitt datron at five
vitt ator frie
videos afferent athie
videos ator attie
vidor at

In [176]:
character_units[28]

'v'

In [138]:
T = 100
from torch.nn.functional import softmax
softmax(single_trial_logits_char[0:1, 8]/100)

/tmp/ipykernel_1672609/1849404952.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax(single_trial_logits_char[0:1, 8]/100)


tensor([[0.0342, 0.0301, 0.0239, 0.0240, 0.0240, 0.0239, 0.0279, 0.0303, 0.0326,
         0.0309, 0.0321, 0.0303, 0.0341, 0.0301, 0.0307, 0.0314, 0.0294, 0.0300,
         0.0314, 0.0319, 0.0309, 0.0297, 0.0314, 0.0285, 0.0334, 0.0308, 0.0308,
         0.0289, 0.0380, 0.0347, 0.0286, 0.0300, 0.0309]])

In [ ]:
ground_truth_arr = []
pred_arr = []
for idx in range(880):
    if idx % 100 == 0:
        print(idx)
    single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
    beam_search_char = decoder_char(single_trial_logits_char)
    beam_search_transcript_char = " ".join(beam_search_char[0][0].words).strip()
    ground_truth_sentence = val_transcripts[idx]
    ground_truth_arr.append(ground_truth_sentence)
    pred_arr.append(beam_search_transcript_char)
    
cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)

In [129]:
values, indices = torch.topk(single_trial_logits_char.squeeze(), dim=-1, k=5)
for time, index in enumerate(indices):
    print(time, [character_units[i] for i in index])


0 ['-', '|', 'i', 'w', 'a']
1 ['-', '|', 'i', 'w', 'a']
2 ['-', '|', 'i', 'w', 'a']
3 ['-', '|', 'i', 'a', 'w']
4 ['-', 'a', '|', 'i', 'o']
5 ['-', 'a', 'i', '|', 'o']
6 ['-', 'a', 'i', '|', 'o']
7 ['-', 'a', 'i', 'o', '|']
8 ['v', 'w', '-', 'f', 'r']
9 ['-', 'v', 'i', 'w', 'r']
10 ['i', 'e', 'a', 'o', '-']
11 ['-', 'i', 'd', 'e', 'r']
12 ['d', '-', 't', 'e', 'i']
13 ['d', 'e', '-', 't', 'i']
14 ['-', 'd', 'e', 't', 'a']
15 ['a', '-', 'o', 't', 'y']
16 ['-', 'a', 'y', '|', 'o']
17 ['|', '-', 'a', 'y', 's']
18 ['|', 'a', '-', 's', 'y']
19 ['-', '|', 'h', 'd', 'r']
20 ['a', 'e', '-', 'f', 'h']
21 ['f', 'a', '-', 'e', 't']
22 ['f', '-', 't', 'v', 'c']
23 ['t', '-', 'f', 'h', 'v']
24 ['t', 'e', '-', 'o', 'f']
25 ['e', 'r', 'o', '-', 'a']
26 ['r', 'n', '-', 'o', 'e']
27 ['r', '-', 'e', 'o', 'n']
28 ['e', '-', 'o', 'n', 'a']
29 ['n', '-', 'e', 'o', 't']
30 ['n', '-', 't', 's', 'k']
31 ['|', '-', 't', 's', 'n']
32 ['|', '-', 'h', 't', 's']
33 ['-', '|', 'h', 'a', 't']
34 ['a', 'o', 'i', 'u', 

In [32]:
torch.std(single_trial_logits_char)

tensor(12.1258)

In [117]:
lm_weights = [2.0]   
word_penalty = [0]
sil_score = [-0.1, -0.2, -0.3]
beam_size = [50]

wer_dict = {}
wer_dict['sil'] = []
wer_dict['lmw'] = []
wer_dict['wp'] = []
wer_dict['bs'] = []
wer_dict['wer'] = []


for sil in sil_score:
    for wp in word_penalty:
      for lmw in lm_weights:
          for bs in beam_size:
          
            decoder_char_lm = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=lexicon_char_file, 
                                beam_size=bs, nbest=50, lm=files.lm, lm_weight=lmw, word_score=wp, sil_score=sil)

            ground_truth_arr = []
            pred_arr = []
        
            for idx in range(880):
                #if idx % 100 == 0:
                #    print(idx)
                single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
                beam_search_char = decoder_char_lm(single_trial_logits_char)
                beam_search_transcript_char = " ".join(beam_search_char[0][0].words).strip()
                ground_truth_sentence = val_transcripts[idx]
                ground_truth_arr.append(ground_truth_sentence)
                pred_arr.append(beam_search_transcript_char)
                
            cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)
            
            
            wer_dict['sil'].append(sil)
            wer_dict['lmw'].append(lmw)
            wer_dict['wp'].append(wp)
            wer_dict['bs'].append(bs)
            wer_dict['wer'].append(wer)
            
            
            print(f"sil score: {sil}, lm weight: {lmw}, word penalty: {wp}, beam size: {bs}, wer: {wer}")


sil score: -0.1, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.26004728132387706
sil score: -0.2, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.259683578832515
sil score: -0.3, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.26041098381523914


In [53]:
phoneme_n_best_list = []
for i in range(len(beam_search_phoneme[0])):
    beam_search_transcript_phoneme= " ".join(beam_search_phoneme[0][i].words).strip()
    phoneme_n_best_list.append(beam_search_transcript_phoneme)
    
print(phoneme_n_best_list)

['void after at five flurry', 'void after at five played', 'void after at five plead', 'void after at five platy', 'void after at five purdie', 'void after at five purdy', 'void after at five ploughed', 'void after at five plowed', 'void after at five lady', 'void after at five leedy', 'void after at five leidy', 'void after at fly purdie', 'void after at fly purdy', 'void after at flye purdie', 'void after at flye purdy', 'void after at flye ploughed', 'void after at flye plowed', 'void after at fly plowed', 'void after at fly ploughed', 'void after at five play', 'void after at five plea', 'void e after at five purdy', 'void ee after at five purdie', 'void e. after at five purdy', 'void e after at five purdie', 'void ee after at five purdy', 'void e. after at five purdie', 'void after at fly lady', 'void after at flye lady', 'void e. after at five plowed', 'void e. after at five ploughed', 'void e after at five plowed', 'void ee after at five plowed', 'void ee after at five ploughed'

In [52]:
char_n_best_list = []
for i in range(len(beam_search_char_lm[0])):
    beam_search_transcript_char = " ".join(beam_search_char_lm[0][i].words).strip()
    char_n_best_list.append(beam_search_transcript_char)
    
print(char_n_best_list)

['vida after at five play', 'vide after at five play', 'vida ater at five play', 'vida after at five plays', 'video after at five play', 'vide ater at five play', 'vida after at five player', 'vide after at five plays', 'vida after en at five play', 'vida after at five ploy', 'vida after n at five play', 'vida after at five pla', 'vida after on at five play', 'vida ator at five play', 'vida after at five playa', 'vida after hat five play', 'vide after at five player', 'vida afferent at five play', 'vida ater at five plays', 'vida atra at five play', 'video ater at five play', 'vida aren at five play', 'vida arn at five play', 'vida datron at five play', 'veda after at five play', 'vide after en at five play', 'vide after at five ploy', 'vide after n at five play', 'video after at five plays', 'vida aron at five play', 'vida tern at five play', 'vida after at five flay', 'vida fern at five play', 'vida aten at five play', 'vide after at five pla', 'vide after on at five play', 'vida ate

In [101]:

single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)


In [56]:
start = time.time()

end = time.time()
print(end-start)

0.013104915618896484


In [16]:
single_trial_logits.shape

torch.Size([1, 61, 41])

In [15]:
decoder.decode_begin()
for t in range(32):
    decoder.decode_step(single_trial_logits[0, t:t + 1, :])
    
decoder.decode_end()
beam_search_result_inc = decoder.get_final_hypothesis()
print(beam_search_result_inc)

[CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 40]), words=['VOID', 'AFTER'], score=522.8263626098633, timesteps=tensor([ 0, 10, 12, 14, 17, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E.', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'EE', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 23, 34, 23, 40]), words=['VOID', 'AFTERNOON'], score=519.06853389

In [59]:
import heapq

def make_new_beam():
  fn = lambda : (NEG_INF, NEG_INF)
  return collections.defaultdict(fn)

def logsumexp(*args):
  """
  Stable log sum exp.
  (Optimized version)
  """
  if not args:
      return NEG_INF

  # <-- 2. Optimized logsumexp
  # Find max in one pass
  a_max = args[0]
  for a in args[1:]:
      if a > a_max:
          a_max = a

  if a_max == NEG_INF:
      return NEG_INF

  # Calculate sum in a second pass
  lsp_sum = 0.0
  for a in args:
      lsp_sum += math.exp(a - a_max)
      
  return a_max + math.log(lsp_sum)

def decode(probs, beam_size=100, blank=0):
  """
  Performs inference for the given output probabilities.
  Arguments:
      probs: The output probabilities (e.g. post-softmax) for each
        time step. Should be an array of shape (time x output dim).
      beam_size (int): Size of the beam to use during inference.
      blank (int): Index of the CTC blank label.
  Returns the output label sequence and the corresponding negative
  log-likelihood estimated by the decoder.
  """
  T, S = probs.shape
  # Convert to log-probabilities once at the beginning
  log_probs = np.log(probs)

  # Elements in the beam are (prefix, (p_blank, p_no_blank))
  # Initialize the beam with the empty sequence, a probability of
  # 1 for ending in blank and zero for ending in non-blank
  # (in log space).
  beam = [(tuple(), (0.0, NEG_INF))]

  for t in range(T): # Loop over time

    # A default dictionary to store the next step candidates.
    next_beam = make_new_beam()

    # <-- 3. Pre-slice probabilities for this time step
    log_probs_t = log_probs[t, :]

    for s in range(S): # Loop over vocab
      p = log_probs_t[s] # Use the pre-sliced array

      # The variables p_b and p_nb are respectively the
      # probabilities for the prefix given that it ends in a
      # blank and does not end in a blank at this time step.
      for prefix, (p_b, p_nb) in beam: # Loop over beam

        # If we propose a blank the prefix doesn't change.
        # Only the probability of ending in blank gets updated.
        if s == blank:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_b = logsumexp(n_p_b, p_b + p, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)
          continue

        # Extend the prefix by the new character s and add it to
        # the beam. Only the probability of not ending in blank
        # gets updated.
        end_t = prefix[-1] if prefix else None
        n_prefix = prefix + (s,)
        n_p_b, n_p_nb = next_beam[n_prefix]
        if s != end_t:
          n_p_nb = logsumexp(n_p_nb, p_b + p, p_nb + p)
        else:
          # We don't include the previous probability of not ending
          # in blank (p_nb) if s is repeated at the end. The CTC
          # algorithm merges characters not separated by a blank.
          n_p_nb = logsumexp(n_p_nb, p_b + p)
          
        next_beam[n_prefix] = (n_p_b, n_p_nb)

        # If s is repeated at the end we also update the unchanged
        # prefix. This is the merging case.
        if s == end_t:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_nb = logsumexp(n_p_nb, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)

    # <-- 4. MAJOR OPTIMIZATION: Use heapq.nlargest instead of full sort
    # Original:
    # beam = sorted(next_beam.items(),
    #         key=lambda x : logsumexp(*x[1]),
    #         reverse=True)
    # beam = beam[:beam_size]
    
    # Optimized:
    beam_items = next_beam.items()
    beam = heapq.nlargest(beam_size,
                          beam_items,
                          key=lambda x: logsumexp(*x[1]))

  # Final selection
  best = beam[0]
  return best[0], -logsumexp(*best[1])

if __name__ == "__main__":
  np.random.seed(3)

  time = 50
  output_dim = 20

  probs = np.random.rand(time, output_dim)
  probs = probs / np.sum(probs, axis=1, keepdims=True)

  labels, score = decode(probs)
  print(labels)
  print("Score {:.3f}".format(score))

(5, 12, 8, 16, 5, 13, 19, 7, 19, 3, 2, 4, 11, 19, 8, 14, 18, 2, 13, 17, 14, 18, 5, 1, 6, 9)
Score 97.032


In [63]:
from scipy.special import softmax
logits = softmax(model_logits[f'arr_750}'],axis=1)
import time

In [65]:
start = time.time()
labels, probs = decode(logits)
end = time.time()
print(end-start)

0.6410787105560303
